# Обработка пропусков в данных, кодирование категориальных признаков, масштабирование данных.

Мы научимся обрабатывать пропуски в данных для количественных (числовых) и категориальных признаков и масштабировать данные. Также мы научимся преобразовывать категориальные признаки в числовые.

### В чем состоит проблема?

- Если в данных есть пропуски, то большинство алгоритмов машинного обучения не будут с ними работать. Даже корреляционная матрица не будет строиться корректно.
- Большинство алгоритмов машинного обучения требуют явного перекодирования категориальных признаков в числовые. Даже если алгоритм не требует этого явно, такое перекодирование возможно стоит попробовать, чтобы повысить качество модели.
- Большинство алгоритмов показывает лучшее качество на масштабированных признаках, в особенности алгоритмы, использующие методы градиентного спуска.


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(style="ticks")

: 

In [ ]:
!pip install numpy
!pip install pandas
!pip install seaborn
!pip install matplotlib

: 

## Загрузка и первичный анализ данных

Используем данные из соревнования [House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques)

In [ ]:
# Будем использовать только обучающую выборку
data = pd.read_csv('data/train.csv', sep=",")

: 

In [ ]:
# размер набора данных
data.shape

: 

In [ ]:
# типы колонок
data.dtypes

: 

In [ ]:
# проверим есть ли пропущенные значения
data.isnull().sum()

: 

In [ ]:
# Первые 5 строк датасета
data.head()

: 

In [ ]:
total_count = data.shape[0]
print('Всего строк: {}'.format(total_count))

: 

# 1. Обработка пропусков в данных

## 1.1. Простые стратегии - удаление или заполнение нулями

In [ ]:
# Удаление колонок, содержащих пустые значения
data_new_1 = data.dropna(axis=1, how='any')
(data.shape, data_new_1.shape)

: 

In [ ]:
# Удаление строк, содержащих пустые значения
data_new_2 = data.dropna(axis=0, how='any')
(data.shape, data_new_2.shape)

: 

In [ ]:
data.head()

: 

In [ ]:
# Заполнение всех пропущенных значений нулями
# В данном случае это некорректно, так как нулями заполняются в том числе категориальные колонки
data_new_3 = data.fillna(0)
data_new_3.head()

: 

## 1.2. "Внедрение значений" - импьютация (imputation)

### 1.2.1. Обработка пропусков в числовых данных

In [ ]:
# Выберем числовые колонки с пропущенными значениями
# Цикл по колонкам датасета
num_cols = []
for col in data.columns:
    # Количество пустых значений 
    temp_null_count = data[data[col].isnull()].shape[0]
    dt = str(data[col].dtype)
    if temp_null_count>0 and (dt=='float64' or dt=='int64'):
        num_cols.append(col)
        temp_perc = round((temp_null_count / total_count) * 100.0, 2)
        print('Колонка {}. Тип данных {}. Количество пустых значений {}, {}%.'.format(col, dt, temp_null_count, temp_perc))

: 

In [ ]:
# Фильтр по колонкам с пропущенными значениями
data_num = data[num_cols]
data_num

: 

In [ ]:
# Гистограмма по признакам
for col in data_num:
    plt.hist(data[col], 50)
    plt.xlabel(col)
    plt.show()

: 

In [ ]:
# Фильтр по пустым значениям поля MasVnrArea 
data[data['MasVnrArea'].isnull()]

: 

In [ ]:
# Запоминаем индексы строк с пустыми значениями
flt_index = data[data['MasVnrArea'].isnull()].index
flt_index

: 

In [ ]:
# Проверяем что выводятся нужные строки
data[data.index.isin(flt_index)]

: 

In [ ]:
# фильтр по колонке
data_num[data_num.index.isin(flt_index)]['MasVnrArea']

: 

Будем использовать встроенные средства импьютации библиотеки scikit-learn - https://scikit-learn.org/stable/modules/impute.html#impute

In [ ]:
data_num_MasVnrArea = data_num[['MasVnrArea']]
data_num_MasVnrArea.head()

: 

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.impute import MissingIndicator

: 

In [ ]:
# Фильтр для проверки заполнения пустых значений
indicator = MissingIndicator()
mask_missing_values_only = indicator.fit_transform(data_num_MasVnrArea)
mask_missing_values_only

: 

С помощью класса [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer) можно проводить импьютацию различными [показателями центра распределения](https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D0%BA%D0%B0%D0%B7%D0%B0%D1%82%D0%B5%D0%BB%D0%B8_%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0_%D1%80%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F)

In [ ]:
strategies=['mean', 'median','most_frequent']

: 

In [ ]:
def test_num_impute(strategy_param):
    imp_num = SimpleImputer(strategy=strategy_param)
    data_num_imp = imp_num.fit_transform(data_num_MasVnrArea)
    return data_num_imp[mask_missing_values_only]

: 

In [ ]:
strategies[0], test_num_impute(strategies[0])

: 

In [ ]:
strategies[1], test_num_impute(strategies[1])

: 

In [ ]:
strategies[2], test_num_impute(strategies[2])

: 

In [ ]:
# Более сложная функция, которая позволяет задавать колонку и вид импьютации
def test_num_impute_col(dataset, column, strategy_param):
    temp_data = dataset[[column]]
    
    indicator = MissingIndicator()
    mask_missing_values_only = indicator.fit_transform(temp_data)
    
    imp_num = SimpleImputer(strategy=strategy_param)
    data_num_imp = imp_num.fit_transform(temp_data)
    
    filled_data = data_num_imp[mask_missing_values_only]
    
    return column, strategy_param, filled_data.size, filled_data[0], filled_data[filled_data.size-1]

: 

In [ ]:
data[['GarageYrBlt']].describe()

: 

In [ ]:
test_num_impute_col(data, 'GarageYrBlt', strategies[0])

: 

In [ ]:
test_num_impute_col(data, 'GarageYrBlt', strategies[1])

: 

In [ ]:
test_num_impute_col(data, 'GarageYrBlt', strategies[2])

: 

### 1.2.2. Обработка пропусков в категориальных данных

In [ ]:
# Выберем категориальные колонки с пропущенными значениями
# Цикл по колонкам датасета
cat_cols = []
for col in data.columns:
    # Количество пустых значений 
    temp_null_count = data[data[col].isnull()].shape[0]
    dt = str(data[col].dtype)
    if temp_null_count>0 and (dt=='object'):
        cat_cols.append(col)
        temp_perc = round((temp_null_count / total_count) * 100.0, 2)
        print('Колонка {}. Тип данных {}. Количество пустых значений {}, {}%.'.format(col, dt, temp_null_count, temp_perc))

: 

### Какие из этих колонок Вы бы выбрали или не выбрали для построения модели?

Класс SimpleImputer можно использовать для категориальных признаков со стратегиями "most_frequent" или "constant".

In [ ]:
cat_temp_data = data[['MasVnrType']]
cat_temp_data.head()

: 

In [ ]:
cat_temp_data['MasVnrType'].unique()

: 

In [ ]:
cat_temp_data[cat_temp_data['MasVnrType'].isnull()].shape

: 

In [ ]:
# Импьютация наиболее частыми значениями
imp2 = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
data_imp2 = imp2.fit_transform(cat_temp_data)
data_imp2

: 

In [ ]:
# Пустые значения отсутствуют
np.unique(data_imp2)

: 

In [ ]:
# Импьютация константой
imp3 = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='!!!')
data_imp3 = imp3.fit_transform(cat_temp_data)
data_imp3

: 

In [ ]:
np.unique(data_imp3)

: 

In [ ]:
data_imp3[data_imp3=='!!!'].size

: 

: 

# 2. Преобразование категориальных признаков в числовые

In [ ]:
cat_enc = pd.DataFrame({'c1':data_imp2.T[0]})
cat_enc

: 

## 2.1. Кодирование категорий целочисленными значениями - [label encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

: 

In [ ]:
le = LabelEncoder()
cat_enc_le = le.fit_transform(cat_enc['c1'])

: 

In [ ]:
cat_enc['c1'].unique()

: 

In [ ]:
np.unique(cat_enc_le)

: 

In [ ]:
le.inverse_transform([0, 1, 2, 3])

: 

## 2.2. Кодирование категорий наборами бинарных значений - [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)

In [ ]:
ohe = OneHotEncoder()
cat_enc_ohe = ohe.fit_transform(cat_enc[['c1']])

: 

In [ ]:
cat_enc.shape

: 

In [ ]:
cat_enc_ohe.shape

: 

In [ ]:
cat_enc_ohe

: 

In [ ]:
cat_enc_ohe.todense()[0:10]

: 

In [ ]:
cat_enc.head(10)

: 

## 2.3. [Pandas get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) - быстрый вариант one-hot кодирования

In [ ]:
pd.get_dummies(cat_enc).head()

: 

In [ ]:
pd.get_dummies(cat_temp_data, dummy_na=True).head()

: 

: 

# 3. Масштабирование данных

Термины "масштабирование" и "нормализация" часто используются как синонимы. Масштабирование предполагает изменение диапазона измерения величины, а нормализация - изменение распределения этой величины.

Если признаки лежат в различных диапазонах, то необходимо их нормализовать. Как правило, применяют два подхода:
- MinMax масштабирование:
$$ x_{новый} = \frac{x_{старый} - min(X)}{max(X)-min(X)} $$

В этом случае значения лежат в диапазоне от 0 до 1.
- Масштабирование данных на основе [Z-оценки](https://ru.wikipedia.org/wiki/Z-%D0%BE%D1%86%D0%B5%D0%BD%D0%BA%D0%B0):
$$ x_{новый} = \frac{x_{старый} - AVG(X) }{\sigma(X)} $$

В этом случае большинство значений попадает в диапазон от -3 до 3.

где $X$ - матрица объект-признак, $AVG(X)$ - среднее значение, $\sigma$ - среднеквадратичное отклонение.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer

: 

## 3.1. [MinMax масштабирование](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)

In [ ]:
sc1 = MinMaxScaler()
sc1_data = sc1.fit_transform(data[['SalePrice']])

: 

In [ ]:
plt.hist(data['SalePrice'], 50)
plt.show()

: 

In [ ]:
plt.hist(sc1_data, 50)
plt.show()

: 

## 3.2. Масштабирование данных на основе [Z-оценки](https://ru.wikipedia.org/wiki/Z-%D0%BE%D1%86%D0%B5%D0%BD%D0%BA%D0%B0) - [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler)

In [ ]:
sc2 = StandardScaler()
sc2_data = sc2.fit_transform(data[['SalePrice']])

: 

In [ ]:
plt.hist(sc2_data, 50)
plt.show()

: 

## 3.3. [Нормализация данных](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html)

In [ ]:
sc3 = Normalizer()
sc3_data = sc3.fit_transform(data[['SalePrice']])

: 

In [ ]:
plt.hist(sc3_data, 50)
plt.show()

: 

# Дополнительные источники
- [Руководство scikit-learn по предобработке данных](https://scikit-learn.org/stable/modules/preprocessing.html)
- [Kaggle Data Cleaning Challenge: Handling missing values (упражнения с пояснениями по обработке пропущенных значений и масштабированию признаков)](https://www.kaggle.com/rtatman/data-cleaning-challenge-handling-missing-values)
- [Краткое руководство по категориальным признакам](https://towardsdatascience.com/encoding-categorical-features-21a2651a065c)
- [Библиотека для сложного кодирования категориальных признаков](https://contrib.scikit-learn.org/categorical-encoding/)